#Téléchargement des packages Python/spaCy

In [ ]:
#!python -m spacy download fr_core_news_sm

Importation des librairies

In [1]:
import time
start = time.time()
import spacy
end = time.time()
print(end - start)
import re
import pandas as pd
import csv
from langdetect import detect

92.42036581039429


In [38]:
# Charger le fichier CSV dans un dataframe
df = pd.read_csv('GooglePlayReviews.csv')

# Fonction pour détecter la langue d'un commentaire
def detect_language(comment):
    try:
        lang = detect(comment)
        return lang
    except:
        return None

# Filtrer les commentaires en français
df_fr = df[df['commentaire'].apply(lambda x: detect_language(x)=='fr')]

df_fr.to_csv('Comments_Filtrés.csv', index=False, header = False)

Lecture du fichier csv

In [39]:
# open the CSV file using the csv module
with open('Comments_Filtrés.csv', newline='',  encoding='utf-8') as csvfile:
    # create a csv reader object
    reader = csv.reader(csvfile)
    # create an empty list to store the strings
    string_list = []
    # loop through each row in the CSV file
    for row in reader:
        # add the string to the list
        #if detect(row[0]) == 'fr':
        string_list.append(row[0])

Exemple de textes

In [ ]:
#Exemple de texte
text = "Selon Wikipédia il l'appelle Laura 😊😊😊(hezignborzingiozrnoivznioznvionzo), La ponctuation est génial l’ensemble des signes qui, dans l’écrit, marquent les divisions et les liaisons des phrases et des membres 'de phrase'. Vous pouvez en apprendre plus sur \"la ponctuation\" en visitant ce site: https://www.larousse.fr/dictionnaires/francais/ponctuation/63717 parce que ce lien donne beaucoup d'informations intéressantes. 😊"
#text = "Fonctionnalités, salut car beau temps aujourd'hui"

Preprocessing

In [35]:
#Etapes du texte Preprocessing
nlp = spacy.load("fr_core_news_sm")
listStopWords = [str(x) for x in nlp.Defaults.stop_words]

Processing the text

In [43]:
def united(text):
    doc =  text.split(" ");
    # Itérer à travers chaque token dans le document
    for token in doc:

        #Remove explanations
        # Si le token est "parce que", "car" ou "puisque", supprimer tous les tokens suivants dans la phrase
        if token in ["parce", "car", "puisque"]:
            doc = doc[:token.index()]
            break
    
    # Supprimer tout ce qui est entre parenthèses
    sentence = ' '.join(doc)
    sentence = re.sub(r'\([^()]*\)', '', sentence)

    print(f"remove explanations : {sentence}")

    
    #Removal of ' and the char that precede it
    sentence = sentence.replace("'", " ")

    print(f"remove leopaul ' : {sentence}")

    words = sentence.split(" ")
    newSentence = ""
    for word in words:
        if len(word) > 1:
            newSentence += word + " "
    sentence = newSentence
    
    print(f"remove quotes : {sentence}")

    sentence = re.sub('http[s]?://\S+', '', sentence)

    print(f"remove links: {sentence}")


    # Regular expressions for phone numbers and email addresses
    phone_regex = r"(?<!\d)(?:\d[ -/\\_\d]*){10}(?!\d)"
    email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"  # matches john.doe@example.com

    # Find phone numbers and email addresses in the sentence
    sentence = re.sub(phone_regex,"", sentence)
    sentence = re.sub(email_regex,"", sentence)
    sentence = re.sub(r'[^\w]', ' ', sentence)
    doc = sentence.split(" ")
    for word in doc:
        if word in listStopWords:
            doc.remove(word)
    
    sentence = ' '.join(doc)

    print(f"remove phone numbers and contacts: {sentence}")

    doc = nlp(sentence)
    #texte = doc.text
    text = ""
    for token in doc:
        if (token.pos_ == "PRON" or token.pos_ == "DET"):
            continue
        else:
            if not token.text.__contains__("'"):
                text += token.text
                text += " "
    sentence = text

    print(f"remove pronoum and determinant : {sentence}")


    doc = nlp(sentence)
    result = []
    for token in doc:
        if not (token.ent_type_ == "ORG" or token.ent_type_ == "PERSON" or token.text.istitle()):
            result.append(token.text)
    sentence =  " ".join(result)
    print(f"remove proper noun : {sentence}")

    return sentence

"""
for string in string_list:
    united_string = united(string)
    string_list_clean.append(united_string)
print(string_list_clean)
#run
#print(united(text))"""
united_string = united(string_list[0])
print(united_string)



remove explanations : Bonjour, Avis aux Amateurs et Professionnels, Depuis une dizaine de jours, et quelques sorties Course à pied, je n'ai plus accès aux données "Moyenne " dans la synthèse proposé par l'application : plus d'allure moyenne, plus de rythme moyen, et plus de vitesse moyenne, etc. Avez une explication et/ou solution. Sinon très bonne application, ou il manque un mode fractionné. Très ergonomique et très intuitive. Merci 😁
remove leopaul ' : Bonjour, Avis aux Amateurs et Professionnels, Depuis une dizaine de jours, et quelques sorties Course à pied, je n ai plus accès aux données "Moyenne " dans la synthèse proposé par l application : plus d allure moyenne, plus de rythme moyen, et plus de vitesse moyenne, etc. Avez une explication et/ou solution. Sinon très bonne application, ou il manque un mode fractionné. Très ergonomique et très intuitive. Merci 😁
remove quotes : Bonjour, Avis aux Amateurs et Professionnels, Depuis une dizaine de jours, et quelques sorties Course pie

In [68]:
testttt = ""
for token in nlp(united_string):
    if (token.pos_ != "SPACE"):
        testttt += token.text
        testttt += " "
print(testttt)

dizaine jours sorties pied ai accès données synthèse proposé application allure moyenne rythme moyen plus de vitesse moyenne explication solution bonne application ou manque mode fractionné ergonomique très intuitive 


In [41]:
print(string_list[0])

Bonjour, Avis aux Amateurs et Professionnels, Depuis une dizaine de jours, et quelques sorties Course à pied, je n'ai plus accès aux données "Moyenne " dans la synthèse proposé par l'application : plus d'allure moyenne, plus de rythme moyen, et plus de vitesse moyenne, etc. Avez une explication et/ou solution. Sinon très bonne application, ou il manque un mode fractionné. Très ergonomique et très intuitive. Merci 😁


#Part 2


In [4]:
united_string

'                dizaine   jours      sorties     pied      n     accès   données   synthèse   proposé     application      d     allure   moyenne      plus   rythme   moyen      plus   de   vitesse   moyenne         explication      solution        bonne   application      manque   mode   fractionné        ergonomique   très   intuitive          '

In [44]:
for token in doc:
    print(token.text)

chat
assoie
sur
canapé
rouge


In [7]:
# Define list of POS patterns to detect
patterns = [
    ("noun noun", ["NOUN", "NOUN"]),
    ("verb noun", ["VERB", "NOUN"]),
    ("adjective noun", ["ADJ", "NOUN"]),
    ("noun conjunction", ["NOUN", "CONJ"]),
    ("noun adjective", ["NOUN", "ADJ"]),
    ("noun noun noun", ["NOUN", "NOUN", "NOUN"]),
    ("verb pronoun noun", ["VERB", "PRON", "NOUN"]),
    ("verb noun noun", ["VERB", "NOUN", "NOUN"]),
    ("verb adjective noun", ["VERB", "ADJ", "NOUN"]),
    ("adjective adjective noun", ["ADJ", "ADJ", "NOUN"]),
    ("noun preposition noun", ["NOUN", "ADP", "NOUN"]),
    ("verb determiner noun", ["VERB", "DET", "NOUN"]),
    ("verb noun preposition noun", ["VERB", "NOUN", "ADP", "NOUN"]),
    ("adjective noun noun noun", ["ADJ", "NOUN", "NOUN", "NOUN"]),
    ("adjective conjunction adjective", ["ADJ", "CONJ", "ADJ"]),
    ("verb preposition adjective noun", ["VERB", "ADP", "ADJ", "NOUN"]),
    ("verb pronoun adjective noun", ["VERB", "PRON", "ADJ", "NOUN"]),
    ("noun conjunction noun noun", ["NOUN", "CONJ", "NOUN", "NOUN"]),
]

In [8]:
# Iterate through the reference patterns and check if there is a match in the sentence
for pattern, pos_tags_pattern in patterns:
    if len(pos_tags_pattern) != len(doc):
        continue  # Skip patterns with different number of words
    match = True
    for i in range(len(doc)):
        if pos_tags_pattern[i] != doc[i].pos_:
            match = False
            break
    if match:
        print(f"Detected pattern '{pattern}' in sentence '{united_string}'")
        break


In [45]:
# Define the input sentence
sentence = "The cat sat on the mat"

# Process the sentence with spaCy
doc = nlp(sentence)

# Iterate through the reference patterns and check if there is a match in the sentence
for pattern, pos_tags_pattern in patterns:
    if len(pos_tags_pattern) != len(doc):
        continue  # Skip patterns with different number of words
    match = True
    for i in range(len(doc)):
        if pos_tags_pattern[i] != doc[i].pos_:
            match = False
            break
    if match:
        print(f"Detected pattern '{pattern}' in sentence '{sentence}'")
        break


In [ ]:
sentenceByPattern = [[] for i in range((18))]

In [70]:
sentence = "chat vert assoie sur canapé rouge"
doc = nlp(testttt)

In [66]:
for token in doc:
    print(token.text)

      
dizaine
jours
   
sorties
pied
   
ai
accès
données
   
synthèse
proposé
application
allure
moyenne
   
rythme
moyen
   
plus
de
vitesse
moyenne
    
explication
solution
   
bonne
application
   
ou
manque
mode
fractionné
   
ergonomique
très
intuitive
  


In [71]:
for i in range(len(doc) - 3):
    if doc[i].pos_ == "NOUN":
        if doc[i + 1].pos_ == "CONJ" and doc[i + 2].pos_ == "NOUN":
            if doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[17].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                break
            else:
                sentenceByPattern[4].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                break
        elif doc[i + 1].pos_ == "PREP" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[11].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
            break
        elif doc[i + 1].pos_ == "NOUN":
            if doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[6].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                break
            else:
                sentenceByPattern[1].append(doc[i].text + " " + doc[i + 1].text)
                break
    elif doc[i].pos_ == "VERB":
        if doc[i + 1].pos_ == "PRON":
            if doc[i + 2].pos_ == "ADJ" and doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[17].append(doc[i].text + " "+ doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                break
            if doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[7].append(doc[i].text +" " + doc[i + 1].text + " " + doc[i + 2].text)
                break
        elif doc[i + 1].pos_ == "PREP" and doc[i + 2].pos_ == "ADJ" and doc[i + 3].pos_ == "NOUN":
            sentenceByPattern[16].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
        elif doc[i + 1].pos_ == "NOUN" :
            if doc[i + 2].pos_ == "PREP" and doc[i + 3].pos_ == "NOUN":
                sentenceByPattern[13].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                break
            elif doc[i + 2].pos_ == "NOUN":
                sentenceByPattern[8].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                break
            else:
                sentenceByPattern[2].append(doc[i].text + " " + doc[i + 1].text)
                break
        elif doc[i + 1].pos_ == "DET" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[12].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
            break
        elif doc[i + 1].pos_ == "ADJ" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[9].append(doc[i].text + " " + doc[i + 1].text + " " +doc[i + 2].text)
            break
    elif doc[i].pos_ == "ADJ" :
        if doc[i + 1].pos_ == "CONJ" and doc[i + 2].pos_ == "ADJ":
            sentenceByPattern[15].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
            break
        elif doc[i + 1].pos_ == "NOUN":
            if doc[i + 2].pos_ == "NOUN":
                if doc[i + 3].pos_ == "NOUN":
                    sentenceByPattern[14].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text + " " + doc[i + 3].text)
                    break
                else:
                    sentenceByPattern[5].append(doc[i].text + " " + doc[i + 1].text + " " + doc[i + 2].text)
                    break
            else:
                sentenceByPattern[3].append(doc[i].text + " " + doc[i + 1].text)
                break
        elif doc[i + 1].pos_ == "ADJ" and doc[i + 2].pos_ == "NOUN":
            sentenceByPattern[10].append(doc[i].text + " " + doc[i + 1].text +" " + doc[i + 2].text)
            break
    else:
        break

In [72]:
sentenceByPattern

[[],
 ['dizaine jours'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]